<a href="https://colab.research.google.com/github/mahiinn62/DLRL_assignment/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
RNN
"""

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.utils import to_categorical
import argparse
from pathlib import Path


SEED = 2025
np.random.seed(SEED)
tf.random.set_seed(SEED)

DEFAULT_TEXT = "The beautiful girl whom I met last time is very intelligent also"

DEFAULT_START = "The beautiful girl "

def build_vocab(text):
    chars = sorted(list(set(text)))
    char_to_idx = {c: i for i, c in enumerate(chars)}
    idx_to_char = {i: c for i, c in enumerate(chars)}
    return chars, char_to_idx, idx_to_char

def create_dataset(text, seq_length):
    chars, char_to_idx, idx_to_char = build_vocab(text)
    sequences = []
    labels = []
    for i in range(len(text) - seq_length):
        seq = text[i : i + seq_length]
        label = text[i + seq_length]
        sequences.append([char_to_idx[ch] for ch in seq])
        labels.append(char_to_idx[label])
    X = np.array(sequences, dtype=np.int32)
    y = np.array(labels, dtype=np.int32)

    X_onehot = to_categorical(X, num_classes=len(chars))
    y_onehot = to_categorical(y, num_classes=len(chars))
    return X_onehot, y_onehot, char_to_idx, idx_to_char

def build_model(seq_length, vocab_size, rnn_units=64):
    model = Sequential([
        SimpleRNN(rnn_units, input_shape=(seq_length, vocab_size), activation="tanh"),
        Dense(vocab_size, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

def sample_with_temperature(preds, temperature=1.0):
    """
    Draw an index from a probability array after applying temperature.
    temperature < 1.0 makes sampling greedier; >1.0 makes it more random.
    """
    preds = np.asarray(preds).astype("float64")
    if temperature <= 0:

        return np.argmax(preds)
    preds = np.log(preds + 1e-9) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds.squeeze(), 1)
    return np.argmax(probas)

def generate_text(model, start_seq, char_to_idx, idx_to_char, length=100, temperature=0.8, seq_length=5):
    """
    Generate text of given length using the trained model.
    If start_seq contains unknown characters, we fallback to a valid seed built
    from the training vocabulary.
    """
    vocab = set(char_to_idx.keys())

    if any(ch not in vocab for ch in start_seq) or len(start_seq) < seq_length:

        print("Provided seed contains unknown chars or is too short. Falling back to a training-safe seed.")

        ordered_vocab = "".join(sorted(vocab))
        start_seq = (ordered_vocab * ((seq_length // len(ordered_vocab)) + 1))[:seq_length]

    generated = start_seq
    for _ in range(length):
        last_seq = generated[-seq_length:]
        x = np.array([[char_to_idx[ch] for ch in last_seq]])
        x_onehot = to_categorical(x, num_classes=len(char_to_idx))
        preds = model.predict(x_onehot, verbose=0)
        next_index = sample_with_temperature(preds[0], temperature=temperature)
        next_char = idx_to_char[next_index]
        generated += next_char
    return generated


def main(args):

    text = args.text if args.text else DEFAULT_TEXT
    seq_length = args.seq_length

    X, y, char_to_idx, idx_to_char = create_dataset(text, seq_length)
    vocab_size = len(char_to_idx)

    print(f"Training on text length {len(text)}, vocab size {vocab_size}, sequences {X.shape[0]}")
    model = build_model(seq_length, vocab_size, rnn_units=args.rnn_units)
    model.summary()


    history = model.fit(X, y, epochs=args.epochs, batch_size=args.batch_size, verbose=1)


    seed = args.seed if args.seed else DEFAULT_START
    generated = generate_text(model, seed, char_to_idx, idx_to_char,
                              length=args.gen_length, temperature=args.temperature, seq_length=seq_length)
    print("\n--- Generated text ---\n")
    print(generated)
    print("\n----------------------\n")

if __name__ == "__main__":
    class Args:
        text = None
        seq_length = 5
        epochs = 60
        batch_size = 16
        rnn_units = 64
        seed = None
        gen_length = 80
        temperature = 0.8

    main(Args)


Training on text length 64, vocab size 21, sequences 59


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 64)             │         5,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 21)             │         1,365 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,869 (26.83 KB)

 Trainable params: 6,869 (26.83 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/60
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.1147 - loss: 3.1117
Epoch 2/60
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1553 - loss: 2.9612
Epoch 3/60
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1553 - loss: 2.8516
Epoch 4/60
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1793 - loss: 2.7495
Epoch 5/60
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2507 - loss: 2.6514
Epoch 6/60
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.3320 - loss: 2.5563
Epoch 7/60
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4357 - loss: 2.4638
Epoch 8/60
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4701 - loss: 2.3738
Epoch 9/60
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4982 - loss: 2.2866
Epoch 10/60
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5169 - loss: 2.2023
Epoch 11/60
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4763 - loss: 2.1210
Epoch 12/60
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4721 - loss: 2.0423
E